# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [114]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from [here](https://drive.google.com/file/d/17oJ9NNbwJ8Wv4Jp-mLFiC59k72Brdyha/view?usp=sharing) and place it in the provided data folder. You will access the data folder with a relative path.
Load `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [115]:
# your code here
orders = pd.read_csv('data/orders.csv',sep=',')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [116]:
# your code here
#Subproblem_1
amount_spend_bycustomer = orders.groupby(['CustomerID'])['amount_spent'].sum().reset_index()
amount_spend_bycustomer.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [119]:
#Subproblem_2

# For my assignment I pick 70-90 % as preferred and > 90% as VIP  
cutoffs = [0,.25,.50,.75,.95,1]
cust_class = ["Low","Med","High","Preferred","VIP"]


amount_spend_bycustomer['Customer_Class'] = pd.qcut(amount_spend_bycustomer["amount_spent"], q = cutoffs,labels=cust_class)
amount_spend_bycustomer.head().sort_values('amount_spent',ascending=False)

,CustomerID,amount_spent,Customer_Class
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Med


In [120]:
results, bin_cuttoffs = pd.qcut(amount_spend_bycustomer["amount_spent"], q = cutoffs,labels=cust_class,retbins=True)
#print(Bin cutoffs)
pd.DataFrame(zip(bin_cuttoffs,cust_class),columns=['Bin Thresholds','Customer Class'])

,Bin Thresholds,Customer Class
0,0.000,Low
1,307.245,Med
2,674.450,High
3,1661.640,Preferred
4,5840.182,VIP


In [ ]:
##subproblem 2

In [121]:
preferred = amount_spend_bycustomer[amount_spend_bycustomer['Customer_Class'] == 'Preferred']
preferred

,CustomerID,amount_spent,Customer_Class
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred
9,12356,2811.43,Preferred
...,...,...,...
4319,18259,2338.60,Preferred
4320,18260,2643.20,Preferred
4328,18272,3078.58,Preferred
4337,18283,2094.88,Preferred


In [122]:
VIP = amount_spend_bycustomer[amount_spend_bycustomer['Customer_Class'] == 'VIP']
VIP

,CustomerID,amount_spent,Customer_Class
0,12346,77183.60,VIP
10,12357,6207.67,VIP
12,12359,6372.58,VIP
50,12409,11072.67,VIP
55,12415,124914.53,VIP
...,...,...,...
4207,18109,8052.97,VIP
4229,18139,8438.34,VIP
4253,18172,7561.68,VIP
4292,18223,6484.54,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [123]:
# your code here

orders[orders['CustomerID'].isin(list(VIP.CustomerID))].groupby('Country')['CustomerID'].count().sort_values(ascending=False)

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
Name: CustomerID, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [124]:
# your code here

orders[(orders['CustomerID'].isin(list(VIP.CustomerID))) | (orders['CustomerID'].isin(list(preferred.CustomerID))) ].groupby('Country')['CustomerID'].count().sort_values(ascending=False)

Country
United Kingdom     221635
Germany              7349
EIRE                 7238
France               6301
Netherlands          2080
Spain                1569
Belgium              1557
Switzerland          1370
Portugal             1093
Australia            1028
Norway               1028
Channel Islands       589
Italy                 507
Finland               504
Cyprus                451
Sweden                273
Japan                 272
Singapore             222
Denmark               217
Israel                214
Iceland               182
Austria               158
Poland                149
Canada                135
Malta                  67
Lebanon                45
Greece                 32
Name: CustomerID, dtype: int64